In [1]:
import json
import pandas as pd
import sqlalchemy as sa
import os

In [5]:
username = ''
password = ''
host = ''
port = ''
database = ''
dialect = ''

database_url = f'{dialect}://{username}:{password}@{host}:{port}/{database}'

In [ ]:
sa.create

In [20]:
engine = sa.create_engine(database_url)
with engine.connect() as conn:
    r = conn.execute(sa.text('SELECT * FROM Developer'))
    data = r.fetchall()

with engine.connect() as conn:
    df = pd.read_sql('SELECT * FROM Developer', conn)

In [ ]:
df

In [ ]:
with open('orders.json', 'r') as rfile:
    #TODO: check if file exists or connect to a database
    _data = json.load(rfile)
_data

In [5]:
null = None
event = {
  "event": {
    "resource": "/inventory-management/inventory",
    "path": "/inventory-management/inventory",
    "httpMethod": "GET",
    "headers": {
      "Accept": "*/*",
      "Accept-Encoding": "gzip, deflate, br",
      "Host": "1zpl4u5btg.execute-api.us-east-2.amazonaws.com",
      "Postman-Token": "8d411abd-b692-4f19-8c89-6c00ef9fd898",
      "User-Agent": "PostmanRuntime/7.42.0",
      "X-Amzn-Trace-Id": "Root=1-671a7d4c-7b03ce6f1cd34f12487861c8",
      "X-Forwarded-For": "140.254.77.242",
      "X-Forwarded-Port": "443",
      "X-Forwarded-Proto": "https"
    },
    "multiValueHeaders": {
      "Accept": ["*/*"],
      "Accept-Encoding": ["gzip, deflate, br"],
      "Host": ["1zpl4u5btg.execute-api.us-east-2.amazonaws.com"],
      "Postman-Token": ["8d411abd-b692-4f19-8c89-6c00ef9fd898"],
      "User-Agent": ["PostmanRuntime/7.42.0"],
      "X-Amzn-Trace-Id": ["Root=1-671a7d4c-7b03ce6f1cd34f12487861c8"],
      "X-Forwarded-For": ["140.254.77.242"],
      "X-Forwarded-Port": ["443"],
      "X-Forwarded-Proto": ["https"]
    },
    "queryStringParameters": null,
    "multiValueQueryStringParameters": null,
    "pathParameters": null,
    "stageVariables": null,
    "requestContext": {
      "resourceId": "k3wftx",
      "resourcePath": "/inventory-management/inventory",
      "httpMethod": "GET",
      "extendedRequestId": "AKiEBGKIiYcEZ7A=",
      "requestTime": "24/Oct/2024:17:01:00 +0000",
      "path": "/Test/inventory-management/inventory",
      "accountId": "585008064068",
      "protocol": "HTTP/1.1",
      "stage": "Test",
      "domainPrefix": "1zpl4u5btg",
      "requestTimeEpoch": 1729789260559,
      "requestId": "45665557-b70e-4962-8594-897780a66031",
      "identity": {
        "cognitoIdentityPoolId": null,
        "accountId": null,
        "cognitoIdentityId": null,
        "caller": null,
        "sourceIp": "140.254.77.242",
        "principalOrgId": null,
        "accessKey": null,
        "cognitoAuthenticationType": null,
        "cognitoAuthenticationProvider": null,
        "userArn": null,
        "userAgent": "PostmanRuntime/7.42.0",
        "user": null
      },
      "domainName": "1zpl4u5btg.execute-api.us-east-2.amazonaws.com",
      "deploymentId": "96mgjy",
      "apiId": "1zpl4u5btg"
    },
    "body": null,
    "isBase64Encoded": False
  },
  "context": "LambdaContext([aws_request_id=eaa5ddda-1612-44fd-aeba-cc6a50774e58,log_group_name=/aws/lambda/toaster-city-inventory,log_stream_name=2024/10/24/[$LATEST]b4057499259649039bb52c2cae11f0d8,function_name=toaster-city-inventory,memory_limit_in_mb=128,function_version=$LATEST,invoked_function_arn=arn:aws:lambda:us-east-2:585008064068:function:toaster-city-inventory,client_context=null,identity=CognitoIdentity([cognito_identity_id=null,cognito_identity_pool_id=null])])"
}


In [ ]:
hash(json.dumps(event))